In [2]:
!pip install langchain
!pip install Openai
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 15.8 MB/s eta 0:00:00


In [5]:
from langchain_openai import ChatOpenAI
from langchain.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

oai = ChatOpenAI()
hf = HuggingFaceEndpoint(repo_id = "google/flan-t5-large")

prompt = PromptTemplate(
    input_variables=["place"],
    template="Best places to visit in {place}"
)

sec_template = PromptTemplate(
    input_variables=["expenses"],
    template="""Given a list of places, please estimate the expenses to visit all of them in local currency and also the days needed {expenses}
    
    Your response:
    """
)

place_chain = LLMChain(llm=oai,prompt=prompt)
expenses_chain = LLMChain(llm=oai,prompt=sec_template)
final_chain = SimpleSequentialChain(chains=[place_chain,expenses_chain],verbose=True)

print(final_chain.run("Brazil"))



Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/claudioed/.cache/huggingface/token
Login successful


> Entering new SimpleSequentialChain chain...
1. Rio de Janeiro - Known for its iconic beaches, vibrant culture, and stunning landscapes, Rio de Janeiro is a must-visit destination in Brazil. Don't miss the famous Christ the Redeemer statue, Copacabana Beach, and Sugarloaf Mountain.

2. Salvador - This historic city in northeastern Brazil is known for its colorful colonial architecture, lively music and dance scene, and rich Afro-Brazilian culture. Explore the Pelourinho district, visit the Mercado Modelo, and enjoy a traditional capoeira performance.

3. Amazon Rainforest - Experience the unparalleled biodiversity of the Amazon Rainforest by taking a guided tour through the jungle, spotting wildlife like monkeys, birds, and caimans,

In [7]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime

rapper_template: str = """You are an American rapper, your job is to come up with\
lyrics based on a given topic

Here is the topic you have been asked to generate a lyrics on:
{input}\
"""

rapper_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=rapper_template)

# creating the rapper chain
rapper_chain: LLMChain = LLMChain(
    llm=oai, output_key="lyric", prompt=rapper_prompt_template)

# verifier
verifier_template: str = """You are a verifier of rap songs, you are tasked\
to inspect the lyrics of rap songs. If they consist of violence and abusive languge\
flag the lyrics. 

Your response should be in the following format, as a Python Dictionary.
lyric: this should be the lyric you received 
Violence_words: True or False

Here is the lyrics submitted to you:
{lyric}\
"""

verified_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["lyric"], template=verifier_template)

# creating the verifier chain
verifier_chain: LLMChain = LLMChain(
    llm=oai, output_key="AI_verified", prompt=verified_prompt_template)


# final output chain
final_template: str = """You are a final quality assurance of a lyrics post.\
Your job will be to accept a lyric and output data in the following format

Your final response should be in the following format, in a Python Dictionary format:
lyric: this should be the lyric you received
Date and time verified: {time_created_and_verified}
Verified by human: {verified_by_human}

Here is the lyric submitted to you:
{AI_verified}\
"""

final_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["AI_verified", "time_created_and_verified", "verified_by_human"], template=final_template)

# creating the verifier chain
final_chain: LLMChain = LLMChain(
    llm=oai, output_key="final_output", prompt=final_prompt_template)


# creating the simple sequential chain
ss_chain: SequentialChain = SequentialChain(
    memory=SimpleMemory(memories={
                        "time_created_and_verified": str(datetime.utcnow()), "verified_by_human": "False"}),
    chains=[rapper_chain, verifier_chain, final_chain],
    # multiple variables
    input_variables=["input"],
    output_variables=["final_output"],
    verbose=True)

# running chain
review = ss_chain.run(input="Gun violence")
print(review)

/var/folders/5p/3r1qqj6j46l12lfvhnlt886c0000gn/T/ipykernel_90502/2295046050.py:64: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time_created_and_verified": str(datetime.utcnow()), "verified_by_human": "False"}),




> Entering new SequentialChain chain...

> Finished chain.
{
    lyric: "Every day I wake up to the news Another shooting, another family lose Bullets flying, innocent lives taken When will the madness end, when will we awaken?",
    Date and time verified: 2024-04-28 21:14:44.694822,
    Verified by human: False
}
